In [224]:
import requests
import numpy as np
from math import log, exp
from scipy.optimize import linear_sum_assignment
import time

In [225]:
headers = {
        "X-RapidAPI-Key": "646b47ee09mshb839bd4c8a546a0p121e42jsn90bf68a43620",
        "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
    }

url = "https://twelve-data1.p.rapidapi.com/price"

def get_price(curr1, curr2):
    if curr1 == curr2:
        return 1
    else:
        try:
            return float(eval(requests.request("GET", url, headers=headers, params={"symbol":f"{curr1}/{curr2}","format":"json","outputsize":"30"}).text)['price'])
        except KeyError:
            return 0

In [226]:
def forex_trade(curr = np.array(['AUD', 'CHF', 'EUR', 'JPY'])):

    M = np.matrix([[0.0 for _ in curr]for _ in curr], dtype=float)
    C = np.matrix([[False for _ in curr]for _ in curr], dtype=bool)

    for r, curr1 in enumerate(curr):
        for c, curr2 in enumerate(curr):
            if C[r, c] == False:
                p = get_price(curr1, curr2)
                if p != 0:
                    M[r, c] = log(float(p))
                    M[c, r] = log(float(1/p))
                else:
                    M[r, c] = 0.0
                    M[c, r] = 0.0
                C[r, c] = C[c, r] = True
        
    
    row_ind, col_ind = linear_sum_assignment(M, maximize=True)

    return M, row_ind, col_ind

In [218]:
ratio_track = []

while True:
    ratio_track.append(forex_trade())
    time.sleep(32)

KeyboardInterrupt: 

In [219]:
M, row_ind, col_ind = forex_trade()

In [220]:
M

matrix([[ 0.        , -0.36201776, -0.39988073,  0.        ],
        [ 0.36201776,  0.        ,  0.        ,  0.        ],
        [ 0.39988073,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [221]:
get_price('AUD', 'CHF')

0

In [223]:
M, row_ind, col_ind = ratio_track[0]
M[row_ind, col_ind].sum()

-0.001425612187942349